<a href="https://colab.research.google.com/github/BarGinger/DIS-Assignment/blob/main/Src/dis_notebook_02_11_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install graphframes
!pip install sparkmeasure==0.24
!pip install matplotlib seaborn
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u422-b05-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col,
    udf,
    row_number,
    countDistinct,
    collect_list,
    struct,
    count,
    sum,
    avg,
    expr,
    percentile_approx,
    max as spark_max,
    explode,
    round,
    rand,
    monotonically_increasing_id,
    array,
    lit,
    broadcast,
    lag
)
import pyspark.sql.functions as F
from sparkmeasure import StageMetrics
from pyspark.sql.types import (
    StringType, IntegerType, BinaryType, DoubleType,
    ArrayType, StructType, StructField, LongType, TimestampType
)
from pyspark.sql import Window
from datetime import datetime, timedelta
from graphframes import GraphFrame
from scipy.sparse import csr_matrix, vstack, hstack
import numpy as np
import pandas as pd
import pickle
import base64
from sparkmeasure import StageMetrics # for resources monitoring
from functools import wraps
import time
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
from operator import truediv
from google.colab import files

In [3]:
# Generate datasets
# Initialize Spark session
spark = SparkSession.builder \
  .appName("PhoneCallsCommunityDetection") \
  .master("local[*]") \
  .config("spark.jars.packages", "ch.cern.sparkmeasure:spark-measure_2.12:0.24,graphframes:graphframes:0.8.2-spark3.1-s_2.12") \
  .config("spark.executor.memory", "20G") \
  .config("spark.driver.memory", "50G") \
  .config("spark.executor.memoryOverhead", "1G") \
  .config("spark.default.parallelism", "100") \
  .config("spark.sql.shuffle.partitions", "10") \
  .config("spark.driver.maxResultSize", "2G") \
  .getOrCreate()


# Initialize StageMetrics
stagemetrics = StageMetrics(spark)

# Optional: Set logging level to reduce verbosity
spark.sparkContext.setLogLevel("WARN")

# Set a checkpoint directory for Spark
spark.sparkContext.setCheckpointDir("/tmp/spark-checkpoints")

In [4]:
# Monitor CPU, Memory and running time
def format_memory(report_memory_output):
  # Initialize a list to store parsed data
    parsed_data = []

    # Iterate through each line in report_memory_output and parse data
    for line in report_memory_output.split('\n'):
        if line.startswith("Stage"):
            # Split the line into parts
            parts = line.split()

            # Extract information
            stage = parts[1]
            metric = parts[2]
            raw_value = int(parts[6])  # Raw value in bytes (integer)

            # Extract formatted value and units
            formatted_value_with_units = " ".join(parts[7:]).replace("(", "").replace(")", "")
            formatted_value, units = formatted_value_with_units.split(" ", 1)

            # Append the extracted information to parsed_data
            parsed_data.append({
                "stageId": stage,
                "memory_metric": metric,
                "memory_raw_value_bytes": raw_value,
                "memory_formatted_value": formatted_value,
                "memory_units": units
            })

    # Create DataFrame
    df = pd.DataFrame(parsed_data)
    return spark.createDataFrame(df)


def track_stage(stage_name):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print(f"Starting {stage_name}")
            stagemetrics.begin()  # Begin collecting metrics for this stage

            result = func(*args, **kwargs)  # Run the actual function

            stagemetrics.end()  # Stop collecting metrics for this stage

            time.sleep(15)

            # Generate metrics DataFrame
            print(f"Completed {stage_name}\n")
            df_metrics_all = stagemetrics.create_stagemetrics_DF()
            df_metrics_agg = stagemetrics.aggregate_stagemetrics_DF()
            # Add stage_name column and join metrics and memory DataFrames
            df_metrics_agg = df_metrics_agg.withColumn("stage_name", pyspark.sql.functions.lit(stage_name))
            df_metrics_agg = df_metrics_agg.withColumn("dataset", pyspark.sql.functions.lit(dataset_file_path))

            df_metrics_agg.show(truncate=False)


            # Generate memory information DataFrame
            # memory_info = stagemetrics.report_memory()
            # df_memory = format_memory(memory_info)


            # df_metrics = df_metrics.join(df_memory, on=['stageId'], how='left')

            # Set write mode based on the stage
            if "Stage 1" in stage_name and clear_csv:
                write_mode = "overwrite"
                header = "true"
            else:
                write_mode = "append"
                header = "true"

            # Write metrics to CSV with appropriate mode and header settings
            df_metrics_agg.coalesce(1).write \
                .mode(write_mode) \
                .option("header", header) \
                .csv(f"{dataset_name}_stage_metrics")

            return result
        return wrapper
    return decorator

In [5]:
# Utils functions

# Convert YYMMDDHHMM to a proper datetime object
def calculate_duration_minutes(start_time, end_time):
  """
  Calculate the duration between two times in minutes.

  Parameters:
  -----------
  start_time : str
      The start time in HH:MM:SS format.
  end_time : str
      The end time in HH:MM:SS format.

  Returns:
  --------
  duration_minutes : float
      The duration between start_time and end_time in minutes.
  """
  start_datetime = convert_to_datetime(start_time)
  end_datetime = convert_to_datetime(end_time)
  duration = end_datetime - start_datetime
  duration_minutes = duration.total_seconds() / 60
  return duration_minutes

def convert_to_datetime(time_str):
  """
  Convert a time string in '%y%m%d%H%M' format to a datetime object.

  Parameters:
  -----------
  time_str : str
      The time string in '%y%m%d%H%M' format.

  Returns:
  --------
  datetime_obj : datetime.datetime
      The datetime object representing the given time string.
  """
  # Use datetime.datetime.strptime to parse the time string
  # This is the correct way to use strptime, avoiding the AttributeError
  return datetime.strptime(str(time_str), '%y%m%d%H%M')

# Define UDF for calculating duration in DDHHMM format
def calculate_duration_string(start_time, end_time):
    start_dt = convert_to_datetime(start_time)
    end_dt = convert_to_datetime(end_time)
    duration = end_dt - start_dt

    days = duration.days
    hours, remainder = divmod(duration.seconds, 3600)
    minutes = remainder // 60
    return f'{days:02d}{hours:02d}{minutes:02d}'

# prompt: print csr_matrix_result pretty
def pretty_print_csr_matrix(csr_matrix_result):
  """Prints a CSR matrix in a readable format."""

  rows, cols = csr_matrix_result.nonzero()
  data = csr_matrix_result.data

  df = pd.DataFrame({
      'Row': rows,
      'Col': cols,
      'Value': data
  })

  print(df)

def create_csr_matrix_from_edges_with_spark(members_df):
    """
    Creates a CSR matrix from a Spark DataFrame based on unique vertices.

    Args:
        members_df: Spark DataFrame with 'community_id' and 'members' columns.

    Returns:
        A CSR matrix.
    """

    # Explode the members array to get each connection in separate rows
    exploded_df = members_df.select(
        "community_id",
        explode("members").alias("member")
    ).select(
        "community_id",
        col("member.Client1").alias("Client1"),
        col("member.Client2").alias("Client2"),
        col("member.duration_minutes").alias("duration_minutes")
    )

    # Get unique clients and create a mapping to indices
    unique_clients = exploded_df.select("Client1").union(exploded_df.select("Client2")).distinct().rdd.flatMap(lambda x: x).collect()
    client_to_index = {client: i for i, client in enumerate(unique_clients)}
    num_clients = len(unique_clients)

    # Extract data for CSR matrix
    rows = exploded_df.select("Client1").rdd.map(lambda row: client_to_index[row[0]]).collect()
    cols = exploded_df.select("Client2").rdd.map(lambda row: client_to_index[row[0]]).collect()
    data = exploded_df.select("duration_minutes").rdd.flatMap(lambda x: x).collect()

    # Create CSR matrix
    csr = csr_matrix((data, (rows, cols)), shape=(num_clients, num_clients))

    return csr

# create csr matrix from given members list
def create_csr_matrix(members, use_weights=False):
    clients = list(set([member['Client1'] for member in members] + [member['Client2'] for member in members]))
    client_index = {client: idx for idx, client in enumerate(clients)}

    row_indices = []
    col_indices = []
    data = []

    for member in members:
        row_indices.append(client_index[member['Client1']])
        col_indices.append(client_index[member['Client2']])
        if use_weights:
            data.append(float(member['duration_minutes']))  # Use duration in minutes as the weight of the edge
        else:
            data.append(1)  # Use 1 for unweighted similarity

    num_clients = len(clients)
    csr = csr_matrix((data, (row_indices, col_indices)), shape=(num_clients, num_clients))

    # Serialize the CSR matrix
    serialized_csr = base64.b64encode(pickle.dumps(csr)).decode('utf-8')
    return serialized_csr

# compare given two csr matrices (each relating to a community) to get similarity score
def compare_weighted_structural_similarity(csr_matrix_1, csr_matrix_2):
    # Deserialize CSR matrices
    csr_1 = pickle.loads(base64.b64decode(csr_matrix_1))
    csr_2 = pickle.loads(base64.b64decode(csr_matrix_2))


    # Align matrix dimensions to the largest size
    max_rows = max(csr_1.shape[0], csr_2.shape[0])
    max_cols = max(csr_1.shape[1], csr_2.shape[1])

    # Pad csr_1 to match max dimensions
    if csr_1.shape[0] < max_rows or csr_1.shape[1] < max_cols:
        csr_1 = vstack([csr_1, csr_matrix((max_rows - csr_1.shape[0], csr_1.shape[1]))]) if csr_1.shape[0] < max_rows else csr_1
        csr_1 = hstack([csr_1, csr_matrix((csr_1.shape[0], max_cols - csr_1.shape[1]))]) if csr_1.shape[1] < max_cols else csr_1

    # Pad csr_2 to match max dimensions
    if csr_2.shape[0] < max_rows or csr_2.shape[1] < max_cols:
        csr_2 = vstack([csr_2, csr_matrix((max_rows - csr_2.shape[0], csr_2.shape[1]))]) if csr_2.shape[0] < max_rows else csr_2
        csr_2 = hstack([csr_2, csr_matrix((csr_2.shape[0], max_cols - csr_2.shape[1]))]) if csr_2.shape[1] < max_cols else csr_2

    # Calculate structural similarity (e.g., using cosine similarity)
    dot_product = csr_1.multiply(csr_2).sum()
    norm_1 = np.sqrt(csr_1.multiply(csr_1).sum())
    norm_2 = np.sqrt(csr_2.multiply(csr_2).sum())
    similarity = dot_product / (norm_1 * norm_2) if norm_1 != 0 and norm_2 != 0 else 0
    return float(similarity)

from pyspark.sql.functions import pandas_udf, PandasUDFType, col, explode, struct
from pyspark.sql.types import BinaryType, StructType, StructField, IntegerType
from scipy.sparse import csr_matrix
import pandas as pd
import pickle
'''Decorator and Function Definition:
The @pandas_udf decorator marks this function as a Pandas UDF (User Defined Function) that will be applied on grouped data.
GROUPED_MAP tells Spark that the function will receive a DataFrame for each group (grouped by community_id).
The schema defines the expected output structure of the function, which is a DataFrame with community_id
and a binary field containing the serialized matrix.
The function converts the connections (edges) between clients into a CSR matrix and serializes it for storage.'''

# Define the schema for the Pandas UDF output
schema = StructType([
    StructField("community_id", IntegerType(), True),
    StructField("csr_matrix", BinaryType(), True)
])

@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def create_csr_matrix_from_edges(members_df):
    """
    Creates a serialized CSR matrix from a Spark DataFrame for each community.

    Args:
        members_df: Spark DataFrame with 'community_id' and 'members' columns.

    Returns:
        DataFrame with 'community_id' and a serialized CSR matrix as binary data.
    """

    # Extract the community ID (assuming it's consistent within the group)
    community_id = members_df['community_id'].iloc[0]
    '''Since each members_df contains data for a single community (due to groupBy operation),
    the function retrieves the community_id from the first row.
    This ID will be included in the output so that each serialized CSR matrix can be linked back
    to its respective community.'''
    # Explode the members array to get each connection in separate rows
    exploded_df = members_df.explode("members").dropna().reset_index(drop=True)
    exploded_df = pd.DataFrame({
        'Client1': exploded_df['members'].apply(lambda x: x['Client1']),
        'Client2': exploded_df['members'].apply(lambda x: x['Client2']),
        'duration_minutes': exploded_df['members'].apply(lambda x: x['duration_minutes'])
    })
    '''Flattening and Extracting Connection Data:
    The members_df contains a column with a list of connections (pairs of clients and call durations).
    The function uses explode to convert this list into individual rows, making it easier to work with each connection.
    It then creates a new DataFrame, exploded_df, with separate columns for Client1, Client2, and duration_minutes
    extracted from the connection data.
    This simplifies further processing by ensuring each row represents a single call between two clients.'''
    # Get unique clients and create a mapping to indices
    unique_clients = pd.concat([exploded_df['Client1'], exploded_df['Client2']]).unique()
    client_to_index = {client: i for i, client in enumerate(unique_clients)}
    num_clients = len(unique_clients)

    # Extract data for CSR matrix
    rows = exploded_df['Client1'].map(client_to_index).values
    cols = exploded_df['Client2'].map(client_to_index).values
    data = exploded_df['duration_minutes'].values #if weight else [1] * len(rows)


    # Create CSR matrix
    csr = csr_matrix((data, (rows, cols)), shape=(num_clients, num_clients))
    '''Serializing the CSR Matrix: The function uses Python’s pickle module to serialize the CSR matrix.
    This converts the matrix into a binary format, allowing it to be stored or transferred efficiently.
    Serialization is necessary because Spark DataFrames cannot directly store complex Python objects like CSR matrices.'''
    # Serialize CSR matrix to binary format
    serialized_csr = pickle.dumps(csr)

    # Return as DataFrame
    return pd.DataFrame({"community_id": [community_id], "csr_matrix": [serialized_csr]})

In [ ]:
## Generate datasets - only run this if datasets folder is empty / does not exists
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from datetime import datetime, timedelta
import random
import os
import shutil
from google.colab import files  # Import for downloading

# def create_spark_session(app_name="CallDatasetGenerator"):
#     """
#     Create or get a Spark session for distributed data processing with optimized settings.
#     """
#     return SparkSession.builder \
#         .appName(app_name) \
#         .config("spark.sql.shuffle.partitions", "100") \
#         .config("spark.default.parallelism", "100") \
#         .config("spark.executor.memory", "2g") \
#         .config("spark.driver.memory", "2g") \
#         .getOrCreate()

def generate_communities(spark, num_communities, community_size_range, density=0.3, extra_factor=1.5):
    """
    Generate isolated communities with controlled sizes and connections.
    Ensures enough connections for the sample count by using an extra factor.
    """
    communities = []
    for community_id in range(num_communities):
        size = random.randint(community_size_range[0], community_size_range[1])
        base_id = community_id * 1000
        community_clients = [(community_id, base_id + i, base_id + j)
                             for i in range(size) for j in range(i + 1, size) if random.random() < density * extra_factor]
        communities.extend(community_clients)

    return spark.createDataFrame(communities, ["community_id", "client1", "client2"])

def generate_call_times(communities_df, calls_per_connection_range, duration_range, base_time, num_samples):
    """
    Generate call start and end times for each client connection, ensuring total number of samples matches `num_samples`.
    """
    calls_df = communities_df.withColumn(
        "num_calls",
        F.expr(f"floor(rand() * ({calls_per_connection_range[1]} - {calls_per_connection_range[0]} + 1)) + {calls_per_connection_range[0]}")
    ).withColumn(
        "call_id", F.monotonically_increasing_id()
    ).withColumn(
        "calls", F.expr("sequence(1, num_calls)")
    ).select("client1", "client2", "call_id", F.explode("calls").alias("call_num"))

    def generate_times():
        start_time = base_time + timedelta(minutes=random.randint(0, 1440))
        duration = random.randint(duration_range[0], duration_range[1])
        end_time = start_time + timedelta(minutes=duration)
        return start_time.strftime('%y%m%d%H%M'), end_time.strftime('%y%m%d%H%M')

    time_udf = F.udf(lambda: generate_times(), "struct<Start_Time:string, End_Time:string>")
    calls_df = calls_df.withColumn("call_times", time_udf())

    # Ensure consistent schema for the final DataFrame
    calls_df = calls_df.select(
        "client1", "client2", calls_df["call_times.Start_Time"].alias("Start_Time"), calls_df["call_times.End_Time"].alias("End_Time")
    )

    # Limit to the specified number of samples
    final_calls_df = calls_df.limit(num_samples)

    # Retry generation if the sample count isn't met
    while final_calls_df.count() < num_samples:
        additional_df = calls_df.limit(num_samples - final_calls_df.count()).select("client1", "client2", "Start_Time", "End_Time")
        final_calls_df = final_calls_df.union(additional_df).limit(num_samples)

    return final_calls_df

# Function to delete all generated datasets
def delete_generated_datasets():
    folder_path = "/content/datasets/"
    deleted_files = []

    if os.path.exists(folder_path):
        # Loop through each item in the folder
        for item in os.listdir(folder_path):
            item_path = os.path.join(folder_path, item)
            # Check if it's a directory and remove it with shutil.rmtree
            if os.path.isdir(item_path):
                shutil.rmtree(item_path)
            else:
                os.remove(item_path)
            deleted_files.append(item)

        # Print the results
        if deleted_files:
            print("Deleted the following items:")
            for item in deleted_files:
                print(item)
        else:
            print("No files found in the folder to delete.")
    else:
        print("The folder does not exist.")

def save_dataset(dataset, filename):
    """
    Save the generated dataset to a temporary directory and then move to the final directory.

    Parameters:
        dataset (DataFrame): The DataFrame to save, containing generated call data.
        filename (str): Base name for the dataset file.
    """
    # Define the directories
    final_dir = "/content/datasets"
    temp_dir = f"{final_dir}/{filename}_temp"
    final_path = os.path.join(final_dir, filename)

    # Create directories if they don't exist
    os.makedirs(final_dir, exist_ok=True)

    # Write to the temporary directory
    dataset.write.mode("overwrite").option("header", "true").csv(temp_dir)

    # Move the content to the final directory
    if os.path.exists(final_path):
        shutil.rmtree(final_path)
    shutil.move(temp_dir, final_path)

    # Clean up by removing the temporary directory
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)

    print(f"Dataset saved as {final_path}")
    return final_path


if __name__ == "__main__":
    delete_generated_datasets()
    # spark = create_spark_session()

    # Define parameter configurations with num_samples
    parameter_sets = [
        {"num_communities": 5, "community_size_range": (3, 5), "calls_per_connection_range": (1, 2), "duration_range": (30, 120), "density": 0.5, "num_samples": 50},
        {"num_communities": 10, "community_size_range": (5, 7), "calls_per_connection_range": (1, 3), "duration_range": (30, 180), "density": 0.4, "num_samples": 100},
        # Add more configurations as needed
    ]

    base_time = datetime(2024, 1, 1)

    for i, params in enumerate(parameter_sets):
        print(f"\nGenerating dataset for configuration {i + 1}: {params}")

        # Generate communities and call times
        communities_df = generate_communities(
            spark,
            num_communities=params["num_communities"],
            community_size_range=params["community_size_range"],
            density=params["density"],
            extra_factor=2  # Generate more potential connections initially
        )
        calls_df = generate_call_times(
            communities_df,
            calls_per_connection_range=params["calls_per_connection_range"],
            duration_range=params["duration_range"],
            base_time=base_time,
            num_samples=params["num_samples"]
        )

        # Save dataset and print information
        filename = f"dataset_config_{i + 1}"
        final_path = save_dataset(calls_df, filename)
        parameter_sets[i]['dataset_name'] = filename
        parameter_sets[i]['csv_filename'] = final_path

    df_datasets = pd.DataFrame(parameter_sets)
    df_datasets.to_csv("dataset_metadata.csv", index=False)
    # Stop Spark session after completing tasks
    spark.stop()

In [11]:
# Read the
@track_stage("Stage 1: Reading the calls dataset")
def read_csv_to_dataframe(file_path= 'toy_dataset.csv'):
  """
  Read dataset from given path into a Spark DataFrame.
  Parameters:
    -----------
    file_path : str
        The name of the given dataset (unigrams or bigrams or both).

    Returns:
    --------
    df_dataset : DataFrame
        A DataFrame of calls with the given dataset info.
  """
  df_dataset = spark.read.csv(file_path, header=True, inferSchema=True)

  # convert start - end times to duration
  # 1st Register the UDFs in Spark
  calculate_duration_minutes_udf = udf(calculate_duration_minutes, DoubleType())
  calculate_duration_string_udf = udf(calculate_duration_string, StringType())

  # 2nd use udfs to add columns for duration in minutes and DDHHMM format
  df_dataset = df_dataset.withColumn('duration_minutes', calculate_duration_minutes_udf(col('Start_Time'), col('End_Time')))
  df_dataset = df_dataset.withColumn('duration_DDHHMM', calculate_duration_string_udf(col('Start_Time'), col('End_Time')))

  print("The following dataframe has been read from the CSV file:")
  df_dataset.show()
  return df_dataset

@track_stage("Stage 2: Preprocessing and creating the graph")
def create_graph_from_dataframe(df_dataset):
  """
  Create graph in GraphFrame from the calls in the current dataset.
  Parameters:
    -----------
    df_dataset : DataFrame
        A DataFrame of calls with the given dataset info.

    Returns:
    --------
    df_dataset : DataFrame
        A DataFrame of calls with the given dataset info.
  """

  # Create Graph using GraphFrames for community detection
  vertices = df_dataset.selectExpr("Client1 as id").union(df_dataset.selectExpr("Client2 as id")).distinct()
  edges = df_dataset.selectExpr("Client1 as src", "Client2 as dst", "duration_minutes as weight")

  # Cache vertices and edges
  vertices.cache()
  edges.cache()

  # Create a GraphFrame
  g = GraphFrame(vertices, edges)

  # Find connected components (communities) using GraphFrames
  connected_components_result = g.connectedComponents()

  # Create a mapping from original community IDs to sequential ones
  community_mapping = connected_components_result.select("component").distinct() \
      .orderBy("component") \
      .withColumn("new_id", row_number().over(Window.orderBy("component"))) \
      .cache()

  # Join the result (community IDs) with the original dataframe and map to new sequential IDs
  df_with_communities = df_dataset.join(connected_components_result, df_dataset['Client1'] == connected_components_result['id'], 'inner') \
      .join(community_mapping, connected_components_result['component'] == community_mapping['component'], 'inner') \
      .drop(connected_components_result['id']) \
      .drop(community_mapping['component']) \
      .withColumnRenamed('new_id', 'community_id')

  # Calculate the number of unique clients (community size) per community
  community_sizes = df_with_communities.select("community_id", "Client1").union(df_with_communities.select("community_id", "Client2")) \
      .distinct() \
      .groupBy("community_id").agg(countDistinct("Client1").alias("community_size"))

  # Merge the community sizes into the main DataFrame
  df_final = df_with_communities.join(community_sizes, 'community_id')

  # Get list of tuples for each community member by considering both Client1 and Client2
  community_members = df_final.select("community_id", "Client1", "Client2", "duration_DDHHMM", "duration_minutes") \
      .distinct() \
      .groupBy("community_id") \
      .agg(collect_list(struct(col("Client1"),
                            col("Client2"),
                            col("duration_DDHHMM"),
                            col("duration_minutes"))).alias("members")) \
      .orderBy("community_id")

  # Show the final DataFrame with community IDs, duration, and community sizes
  print("\nFinal DataFrame with Sequential Community IDs:")
  df_final.select('Client1',
                  'Client2',
                  'duration_DDHHMM',
                  'duration_minutes',
                  'community_id',
                  'community_size') \
      .orderBy("community_id") \
      .show()

  # Show the list of community members as tuples
  print("\nCommunity Members with Sequential IDs:")
  community_members.show(truncate=False)

  # Save results to CSV files
  # Save the main analysis results
  df_final.select('Client1',
                  'Client2',
                  'duration_DDHHMM',
                  'duration_minutes',
                  'community_id',
                  'community_size') \
      .orderBy("community_id") \
      .write.mode("overwrite").option("header", "true") \
      .csv(f"{dataset_name}_community_analysis_results")

  # Save community members in a flattened format
  df_final.select('community_id',
                  'Client1',
                  'Client2',
                  'duration_DDHHMM',
                  'duration_minutes') \
      .distinct() \
      .orderBy("community_id") \
      .write.mode("overwrite").option("header", "true") \
      .csv(f"{dataset_name}_community_members_results")

  # Optionally, if you want to save additional community statistics
  community_stats = df_final.groupBy('community_id') \
      .agg(
          countDistinct('Client1', 'Client2').alias('unique_members'),
          count('*').alias('total_calls'),
          sum('duration_minutes').alias('total_duration_minutes'),
          avg('duration_minutes').alias('avg_call_duration'),
          percentile_approx('duration_minutes', 0.25).alias('duration_25th_percentile'),
          percentile_approx('duration_minutes', 0.5).alias('median_call_duration'),
          percentile_approx('duration_minutes', 0.75).alias('duration_75th_percentile')
      ) \
      .orderBy('community_id')

  community_stats.write.mode("overwrite") \
      .option("header", "true") \
      .csv(f"{dataset_name}_community_statistics_results")

  print("This is the community stats:")
  community_stats.show(truncate=False)
  return df_final, community_members, community_stats

# Create CSR adjacency matrices for each community and serialize them
@track_stage("Stage 3: Creating CSR matrices")
def format_members_to_csr_matrix(community_members):
  """
  Create CSR adjacency matrices for each community and serialize them.

  Parameters:
    community_members: Dataframe
    A dataframe of a specific community's members
  """
  # Convert the collected list of Row objects to a list of dictionaries before passing to UDF
  schema = StructType([
      StructField("Client1", StringType(), True),
      StructField("Client2", StringType(), True),
      StructField("duration_DDHHMM", StringType(), True),
      StructField("duration_minutes", DoubleType(), True)
  ])
  convert_members_udf = udf(lambda members: [member.asDict() for member in members], ArrayType(schema))
  community_members = community_members.withColumn("members_dict", convert_members_udf(col("members")))
  #Register UDF to create and serialize CSR matrices (both unweighted and weighted)
  create_csr_unweighted_udf = udf(lambda members: create_csr_matrix(members, use_weights=False), StringType())
  create_csr_weighted_udf = udf(lambda members: create_csr_matrix(members, use_weights=True), StringType())

  # Add CSR matrix representations (unweighted and weighted) to each community
  community_members = community_members.withColumn("csr_matrix_unweighted", create_csr_unweighted_udf(col("members_dict")))
  community_members = community_members.withColumn("csr_matrix_weighted", create_csr_weighted_udf(col("members_dict")))

  community_members.show(truncate=False)

  # Print some information about the matrix
  # print(f"CSR Matrix shape: {csr_matrix_result.shape}")
  # print(f"Number of non-zero elements: {csr_matrix_result.nnz}")
  # pretty_print_csr_matrix(csr_matrix_result)

  return community_members

from pyspark.sql import functions as F
from itertools import combinations

def calculate_similarities(subgroup_community_members):
  """
  Comparing CSR matrices to detect similarity
  """

  # Register UDF to compare structural similarity
  compare_structural_similarity_udf = udf(lambda csr_1, csr_2: compare_weighted_structural_similarity(csr_1, csr_2), DoubleType())
  compare_weighted_similarity_udf = udf(lambda csr_1, csr_2: compare_weighted_structural_similarity(csr_1, csr_2), DoubleType())

  # Cross join to compare each pair of communities and calculate both similarities
  cross_joined = subgroup_community_members.alias("a").crossJoin(subgroup_community_members.alias("b")) \
      .filter(col("a.community_id") < col("b.community_id")) \
      .withColumn("unweighted_similarity_score", compare_structural_similarity_udf(col("a.csr_matrix_unweighted"), col("b.csr_matrix_unweighted"))) \
      .withColumn("weighted_similarity_score", compare_weighted_similarity_udf(col("a.csr_matrix_weighted"), col("b.csr_matrix_weighted")))

  # Add combined similarity score (50/50 importance)
  cross_joined = cross_joined.withColumn("combined_similarity_score",
                                        0.5 * col("unweighted_similarity_score") + 0.5 * col("weighted_similarity_score"))

  # Show the similarity scores between communities
  # Rename and select columns to create a new DataFrame for writing
  df_to_export = cross_joined.select(
      col("a.community_id").alias("community_id_1"),
      col("b.community_id").alias("community_id_2"),
      round(col("unweighted_similarity_score"), 2).alias("unweighted_similarity_score"),
      round(col("weighted_similarity_score"), 2).alias("weighted_similarity_score"),
      round(col("combined_similarity_score"), 2).alias("combined_similarity_score")
  )

  # Display the DataFrame
  print("these are the groups:")
  df_to_export.orderBy(["community_id_1", "community_id_2"]).show(truncate=False)

  # Write the new DataFrame to CSV
  print("exporting to csv file")
  df_to_export.write.mode("overwrite").option("header", "true").csv("groups_found")

  return cross_joined

# Function to create similarity-based subgroups by comparing multiple columns
@track_stage("Stage 4: Calculate similarities between communities")
def create_similarity_subgroups(df, columns, tolerances):
    """
    Create similarity-based subgroups based on specified columns and tolerances, then apply a custom function.

    Parameters:
        df (DataFrame): The Spark DataFrame with community data.
        columns (list): List of column names to consider for similarity.
        tolerances (dict): Dictionary specifying the tolerance (± range) for each column.
    """
    # Collect the DataFrame into a list of rows
    communities = df.collect()

    # Initialize a list to store similarity groups
    similarity_groups = []
    df_groups = None

    # Compare each pair of communities
    for i, j in combinations(range(len(communities)), 2):
        community_i = communities[i]
        community_j = communities[j]

        # Check if the communities are similar based on all specified columns and tolerances
        is_similar = all(
            abs(community_i[column] - community_j[column]) <= tolerances[column]
            for column in columns
        )

        # If they are similar, add them to the same group
        if is_similar:
            found_group = False
            for group in similarity_groups:
                if community_i.community_id in group or community_j.community_id in group:
                    group.add(community_i.community_id)
                    group.add(community_j.community_id)
                    found_group = True
                    break
            if not found_group:
                similarity_groups.append({community_i.community_id, community_j.community_id})

    # Create a DataFrame for each subgroup and apply the custom function
    for group in similarity_groups:
        subgroup_df = df.filter(F.col("community_id").isin(group))
        subgroup_cross_joined = calculate_similarities(subgroup_df)
        # Initialize or append to df_groups
        if df_groups is None:
            df_groups = subgroup_cross_joined
        else:
            df_groups = df_groups.union(subgroup_cross_joined)


    # export all found groups
    if df_groups:
      df_groups.write \
                  .mode("overwrite") \
                  .option("header", header) \
                  .csv(f"{dataset_name}_df_groups.csv")
    else:
      print("No groups found!")
    return df_groups

In [16]:
# read dataset dataframe, and iterate over each one to create communities and form groups
df_datasets = pd.read_csv("dataset_metadata.csv")
print("These are the found datasets")
df_datasets.head(10)

# set global params
clear_csv = False
dataset_file_path = "toy_dataset.csv"
dataset_name = "toy_dataset"

for i, dataset in df_datasets.iterrows():
  print(f"Starting to process {i+1} dataset with the following params: \n{dataset}")
  # step 1 - read the dataset
  dataset_file_path = dataset["csv_filename"]
  # Get the base name from the path (e.g., 'file.txt' from '/path/to/file.txt')
  basename =  os.path.basename(dataset_file_path)
  # Split the filename and extension
  dataset_name = os.path.splitext(basename)[0]
  clear_csv = i == 0 # only clear the if this is the 1st dataset
  df_dataset = read_csv_to_dataframe(dataset_file_path)

  # step 2 - preprocess (convert to duartion in min, create grpah, and find commutnies)
  df_final, community_members, community_stats = create_graph_from_dataframe(df_dataset)

  # step 3 - create CSR matrix for each communite
  # csr_matrix_result = format_members_to_csr_matrix(community_members)
  # Use the function to generate a serialized CSR matrix for each community and show the results
  result = community_members.groupBy("community_id").apply(create_csr_matrix_from_edges)
  result.show(truncate=False)

  # step 4 - calculate similarities between communties for find groups
  # Define the columns and tolerances for similarity-based grouping
  columns = ['unique_members']
  tolerances = {'unique_members': 50}

  # Apply the similarity-based grouping and custom function
  cross_joined = create_similarity_subgroups(csr_matrix_result, columns, tolerances)
  # cross_joined = calculate_similarities(csr_matrix_result)

# # end the current spark session
# spark.stop()

These are the found datasets
Starting to process 1 dataset with the following params: 
num_communities                                                5
community_size_range                                      (3, 5)
calls_per_connection_range                                (1, 2)
duration_range                                         (30, 120)
density                                                      0.5
num_samples                                                   50
dataset_name                                    dataset_config_1
csv_filename                  /content/datasets/dataset_config_1
Name: 0, dtype: object
Starting Stage 1: Reading the calls dataset
The following dataframe has been read from the CSV file:
+-------+-------+----------+----------+----------------+---------------+
|client1|client2|Start_Time|  End_Time|duration_minutes|duration_DDHHMM|
+-------+-------+----------+----------+----------------+---------------+
|      0|      1|2401011207|2401011302|           

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")



Final DataFrame with Sequential Community IDs:
+-------+-------+---------------+----------------+------------+--------------+
|Client1|Client2|duration_DDHHMM|duration_minutes|community_id|community_size|
+-------+-------+---------------+----------------+------------+--------------+
|      1|      2|         000050|            50.0|           1|             3|
|      1|      2|         000128|            88.0|           1|             3|
|      1|      2|         000104|            64.0|           1|             3|
|      1|      2|         000141|           101.0|           1|             3|
|      0|      1|         000055|            55.0|           1|             3|
|      0|      2|         000200|           120.0|           1|             3|
|      0|      2|         000031|            31.0|           1|             3|
|      0|      1|         000148|           108.0|           1|             3|
|      0|      2|         000154|           114.0|           1|             3|
|   

PySparkValueError: unique_members

In [14]:
 # step 4 - calculate similarities between communties for find groups
# Define the columns and tolerances for similarity-based grouping
columns = ['unique_members']
tolerances = {'unique_members': 50}

# Apply the similarity-based grouping and custom function
cross_joined = create_similarity_subgroups(community_stats, columns, tolerances)

Starting Stage 4: Calculate similarities between communities


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `a`.`csr_matrix_unweighted` cannot be resolved. Did you mean one of the following? [`a`.`community_id`, `a`.`avg_call_duration`, `b`.`community_id`, `b`.`avg_call_duration`, `a`.`total_calls`].;
'Project [community_id#27283, unique_members#29308L, total_calls#29310L, total_duration_minutes#29312, avg_call_duration#29314, duration_25th_percentile#29316, median_call_duration#29318, duration_75th_percentile#29320, community_id#34189, unique_members#34161L, total_calls#34162L, total_duration_minutes#34163, avg_call_duration#34164, duration_25th_percentile#34165, median_call_duration#34166, duration_75th_percentile#34167, <lambda>('a.csr_matrix_unweighted, 'b.csr_matrix_unweighted)#34206 AS unweighted_similarity_score#34207]
+- Filter (community_id#27283 < community_id#34189)
   +- Join Cross
      :- SubqueryAlias a
      :  +- Filter community_id#27283 IN (1,2,3,4)
      :     +- Sort [communit

In [ ]:
# plot the resources usgae of current dataset
df_monitor = spark.read.csv("stage_metrics",  header=True)
df_monitor = df_monitor.orderBy('stage_name')
print(f"count: {df_monitor.count()}")
df_monitor.show(truncate=False)

# Convert the Spark DataFrame to a Pandas DataFrame for plotting
pdf_monitor = df_monitor.toPandas()

# Select the columns of interest for plotting
columns_of_interest = ['stage_name', 'numTasks', 'stageDuration', 'peakExecutionMemory', 'executorCpuTime']
pdf_plot = pdf_monitor[columns_of_interest].copy()  # Create a copy to avoid the warning

# Format memory and time measurements using .loc
pdf_plot.loc[:, 'peakExecutionMemory'] = pdf_plot['peakExecutionMemory'].astype(float) / (1024**3)  # Bytes to GB
pdf_plot.loc[:, 'stageDuration'] = pdf_plot['stageDuration'].astype(float) / (1000 * 60)  # ms to minutes
pdf_plot.loc[:, 'executorCpuTime'] = pdf_plot['executorCpuTime'].astype(float) / (1000 * 60)  # ms to minutes
pdf_plot.loc[:, 'numTasks'] = pdf_plot['numTasks'].astype(int)

# Extract stage numbers from stage_name
pdf_plot['stage_number'] = pdf_plot['stage_name'].apply(lambda x: int(re.search(r'\d+', x).group()))

# Melt the DataFrame for easier plotting with Seaborn
pdf_plot_melted = pd.melt(pdf_plot, id_vars=['stage_name', 'stage_number'], var_name='Metric', value_name='Value')

fig = plt.figure(figsize=(10, 8))
# Create the plot with facets, stage numbers as x-axis, and legend with full stage names
g = sns.FacetGrid(pdf_plot_melted, col='Metric',
                   height=6, aspect=1.5,
                  col_wrap=2, sharey=False, sharex=False)
g.map(sns.barplot, 'stage_number', 'Value', palette='hls', hue='stage_name',
      data=pdf_plot_melted, dodge=False)  # Pass data argument
g.set_xticklabels(pdf_plot['stage_number'].unique(), size=16)
g.set_titles("{col_name}", size=18)
g.fig.suptitle('Spark Stage Metrics', y=1.02, size=18)
g.add_legend(loc='upper right', bbox_to_anchor=(1.2, 0.92))
g.legend.set_title('Stage Names', prop={'weight': 'bold', 'size': 22})  # Add title and format it
for text in g.legend.get_texts():
    text.set_fontsize(18)  # Legend label font size# Set y-axis labels with units and add x-axis label
for ax in g.axes.flat:
    metric = ax.get_title()
    if metric == 'peakExecutionMemory':
        ax.set_ylabel('Peak Execution Memory (GB)', size=18)
    elif metric in ('stageDuration', 'executorCpuTime'):
        ax.set_ylabel('Time (minutes)', size=18)
    else:
        ax.set_ylabel(metric, size=18)

    ax.set_xlabel('Stage Number', size=18)  # Change x-axis label to "Stage Number"

g.set_yticklabels(fontsize=18)
plt.tight_layout()
plt.show()